### 7.2 Veri seti hazirlanmasi ve Ön işleme

In [1]:
"""
problem tanimi: lstm ile metin turetme
"""

'\nproblem tanimi: lstm ile metin turetme\n'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter # kelime frekanslarini hesaplamak icin 
from itertools import product # grid search icin kombinasyon olusturmak

# veri yukleme ve on isleme (preprocessing)
# urun yorumlari
text = """Bu ürün beklentimi fazlasıyla karşıladı.  
Malzeme kalitesi gerçekten çok iyi.  
Kargo hızlı ve sorunsuz bir şekilde elime ulaştı.  
Fiyatına göre performansı harika.  
Kesinlikle tavsiye ederim ve öneririm!"""

# veri on ısleme: 
# noktalama isaretlerinden kurtul, 
# kucuk harf donusumu
# kelimeleri bol

words = text.replace(".", "").replace("!","").lower().split()

# kelime frekanslarini hesapla ve indeksleme olustur
word_counts = Counter(words)
vocab = sorted(word_counts, key=word_counts.get, reverse = True) # kelime frekansini buyukten kucuge sirala
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

# egitim verisi hazirlama
data = [(words[i], words[i+1]) for i in range(len(words) - 1)]

#### 7.3 Modelin Oluşturulması

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):

        super(LSTM, self).__init__()# bir ust sinifin constructor ini cagirma
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # embedding katmani
        self.lstm = nn.LSTM(embedding_dim, hidden_dim) # LSTM katmani 
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        """
        input -> embedding -> lstm -> fc -> output
        """
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x.view(1,1,-1))
        output = self.fc(lstm_out.view(1,-1))

        return output
        
# model = LSTM(len(vocab), embedding_dim=8, hidden_dim=32)
    

#### 7.4 Hiperparametre Ayarlanmasi

In [ ]:
def prepare_squence(seq, to_ix):
    return torch.tensor([to_ix[w] for w in seq], dtype = torch.long)

# hyperparameter tuning kombinasyonlrini belirle 
embedding_sizes = [8,16]
hidden_sizes = [32, 64]
learning_rates = [0.01, 0.005]
best_loss = float("inf") # en buyuk kayip degerini saklamk icin bir degisken 
best_params = {} # en iyi parametrelerii saklamak icin bos bir dictionary 

print("Hyperparameter tuning basliyor ...")

# grid search 
for emb_size, hidden_size, lr in product(embedding_sizes, hidden_sizes, learning_rates):
    print(f"Deneme: Embedding: {emb_size}, Hidden: {hidden_size}, learning_rate: {lr}")
    
    # modeli tanimla 
    model = LSTM(len(vocab), emb_size, hidden_size) # secilen params ile model olsutur 
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)

    epochs = 50
    total_loss = 0
    for epoch in range(epochs):
        epoch_loss = 0 # epoch baslangıcnda kaybi sıfırlayalım 
        for word, next_word in data:
            model.zero_grad() # gradyanlari sıfırla 
            input_tensor = prepare_squence([word], word_to_ix) #girdiyi tensor e cevir 
            target_tensor  = prepare_squence([next_word], word_to_ix) #3 hedef kelimeyi tensore cevir 
            output = model(input_tensor) # prediction 
            loss = loss_func(output, target_tensor)
            loss.backward() # geri yayilim islmei uygula 
            optimizer.step() # parametreleri guncelle 
            epoch_loss += loss.item()

        if epoch % 10 == 0 : 
            print (f"Epoch: { epoch}, Loss: { epoch_loss:.5f}")
        total_loss = epoch_loss

    # en iyi modeli kaydet 
    if total_loss < best_loss:
        best_loss = total_loss
        best_params = { "embedding_dim": emb_size, "hidden_dim": hidden_size, "learning_rate": lr} 
    print()
print(f"Best params: {best_params}")

Hyperparameter tuning basliyor ...
Deneme: Embedding: 8, Hidden: 32, learning_rate: 0.01
Epoch: 0, Loss: 86.69640
Epoch: 10, Loss: 3.77524
Epoch: 20, Loss: 2.30615
Epoch: 30, Loss: 2.02073
Epoch: 40, Loss: 1.89542

Deneme: Embedding: 8, Hidden: 32, learning_rate: 0.005
Epoch: 0, Loss: 85.57802
Epoch: 10, Loss: 13.83929
Epoch: 20, Loss: 3.61550
Epoch: 30, Loss: 2.47080
Epoch: 40, Loss: 2.10286

Deneme: Embedding: 8, Hidden: 64, learning_rate: 0.01
Epoch: 0, Loss: 86.64381
Epoch: 10, Loss: 2.72745
Epoch: 20, Loss: 2.09292
Epoch: 30, Loss: 1.90552
Epoch: 40, Loss: 1.81623

Deneme: Embedding: 8, Hidden: 64, learning_rate: 0.005
Epoch: 0, Loss: 85.89699
Epoch: 10, Loss: 5.45951
Epoch: 20, Loss: 2.43310
Epoch: 30, Loss: 2.04392
Epoch: 40, Loss: 1.89482

Deneme: Embedding: 16, Hidden: 32, learning_rate: 0.01
Epoch: 0, Loss: 85.73782
Epoch: 10, Loss: 3.15460
Epoch: 20, Loss: 2.16247
Epoch: 30, Loss: 1.94681
Epoch: 40, Loss: 1.85349

Deneme: Embedding: 16, Hidden: 32, learning_rate: 0.005
Epoch

#### 7.5 Model Egitimi

In [9]:
# lstm training

final_model = LSTM(len(vocab), best_params['embedding_dim'], best_params['hidden_dim'])
optimizer = optim.Adam(final_model.parameters(), lr = best_params['learning_rate'])
loss_function = nn.CrossEntropyLoss() # entropi kayip fonksiyonu

print("Final model training")
epochs = 100
for epoch in range(epochs):
    epoch_loss = 0 
    for word, next_word in data:
        final_model.zero_grad()
        input_tensor = prepare_squence([word], word_to_ix)
        target_tensor = prepare_squence([next_word], word_to_ix)
        output = final_model(input_tensor)
        loss = loss_function(output, target_tensor)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Final Model Epoch: {epoch}, Loss: {epoch_loss:.5f}")


Final model training
Final Model Epoch: 0, Loss: 85.57164
Final Model Epoch: 10, Loss: 3.62675
Final Model Epoch: 20, Loss: 2.20463
Final Model Epoch: 30, Loss: 1.95314
Final Model Epoch: 40, Loss: 1.84174
Final Model Epoch: 50, Loss: 1.78036
Final Model Epoch: 60, Loss: 1.73406
Final Model Epoch: 70, Loss: 1.70200
Final Model Epoch: 80, Loss: 1.67694
Final Model Epoch: 90, Loss: 1.65515


#### 7.6 Modelin Test edilmesi ve Degerlendirilmesi

In [10]:
# test ve degerlendirme

# kelime tahmini fonksiyonu: baslangic kelimesi ve n adet kelime uretmesini sagla
def predict_sequence(start_word, num_words):
    current_word = start_word # suanki kelime baslangic kelimesi olarak ayarlanir
    output_sequence = [current_word] # cikti dizisi 
    
    for _ in range(num_words): # belirtilen sayida kelime tahmini
        with torch.no_grad(): # gradyan hesaplamasi yapmadan
            input_tensor = prepare_squence([current_word], word_to_ix) # kelime -> tensor
            output = final_model(input_tensor)
            predicted_idx = torch.argmax(output).item() # en yuksek olasiliga sahip kelimenin indexi
            predicted_word = ix_to_word[predicted_idx] # indekse karsilik gelen kelimeyi return eder
            output_sequence.append(predicted_word)
            current_word = predicted_word # bir sonraki tahmin icin mevcut kelimeleri guncelle
    return output_sequence # tahmin edilen kelime dizisi return edilir.
   
start_word = "ve"           
num_predictions = 2
predicted_sequence = predict_sequence(start_word, num_predictions)
print(" ".join(predicted_sequence))

ve sorunsuz bir
